<h1 align=center><font size = 5>Análise de Dados com Python</font></h1>

<h2>Sumário</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#data_acquisition">Aquisição de Dados</a></li>
    <li><a href="#basic_insight">Visualização Básica do Conjunto de Dados</a></li>
    <li><a href="#data-transformation">Transformação dos Dados</a></li>
    <li><a href="#data_standardization">Padronização dos Dados</a></li>
    <li><a href="#data_normalization">Normalização dos Dados</a></li>
</ul>
</div>
 
<hr>

In [ ]:
# Bibliotecas necessárias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

<h1 id="data_acquisition">Aquisição dos Dados</h1>
<h3>Contexto</h3>
<p>
Este conjunto de dados consiste de dados do Anuário Automotivo de 1985 da Ward. Os dados são oriundos de:

Fontes:

1) Especificações de importação de carros e caminhões de 1985, Anuário Automotivo de 1985 da Ward.
2) Manuais pessoais  de automóveis, Escritório de Serviços de Seguro, 160 Water Street, New York, NY 10038
3) Relatórios de Seguros de Colisão, Instituto de Seguros para Segurança Rodoviária, Watergate 600, Washington, DC 20037

<h3>Conteúdo</h3>
Este conjunto de dados consiste de três tipos de entidades: (a) a especificação de um atomóvel em termos de várias características, (b) sua classificação de risco, (c) sua depreciação em uso comparado com outros carros. A segunda classificação corresponde ao grau em que o automóvel é mais arriscado do que seu preço indica. Os carros recebem inicialmente um fator de risco associado ao seu preço. Então, se for mais arriscado (ou menos), este fator é ajustado movendo-o para cima (ou para baixo) na escala. Os atuários chamam esse processo de "simbolização". Um valor de +3 indica que o automóvel é arriscado, -3 que provavelmente é bastante seguro.

O terceiro fator é a depreciação média relativa por ano de veículo segurado. Este valor é normalizado para todos os automóveis dentro de uma determinada classificação de tamanho (pequeno de duas portas, peruas, esportes/especialidades, etc...) e representa a perda média por carro por ano.

<ul>
    <li>Fonte: <a href="https://www.kaggle.com/toramky/automobile-dataset">https://www.kaggle.com/toramky/automobile-dataset</a></li>
    <li>Formato: csv</li>
</ul>

In [ ]:
data_path = 'data/Automobile_data.csv'
data = pd.read_csv(data_path, na_values='?')
data.head(7)

<h1 id="basic_insight">Visualização Básica do Conjunto de Dados</h1>

In [ ]:
# Informações básicas
df = data.copy()
df.info()

In [ ]:
# Tamanho do conjunto de dados
df.shape

In [ ]:
df.columns

In [ ]:
# Estatísticas básicas para valores numéricos
df.describe()

In [ ]:
# Descrição completa dos dados
df.describe(include="all")

In [ ]:
df.nunique()

In [ ]:
# Visualização da distribuição das características binárias
fig, axs = plt.subplots(2, 2, figsize=(6,4))
axs[0, 0].pie(x=df['fuel-type'].value_counts(), labels=list(df['fuel-type'].value_counts().index), explode = (0, 0.1))
axs[0, 0].set_title("Fuel Type")
axs[0, 1].pie(x=df['aspiration'].value_counts(), labels=list(df['aspiration'].value_counts().index), explode = (0, 0.1))
axs[0, 1].set_title("Aspiration")
axs[1, 0].pie(x=df['num-of-doors'].value_counts(), labels=list(df['num-of-doors'].value_counts().index), explode = (0, 0.1))
axs[1, 0].set_title("Number of doors")
axs[1, 1].pie(x=df['engine-location'].value_counts(), labels=list(df['engine-location'].value_counts().index), explode = (0, 0.1))
axs[1, 1].set_title("Engine location")
plt.show()

In [ ]:
# Distribuição das montadoras
plt.figure(figsize=(8,5))
sns.countplot(df.make)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Distribuição dos modelos
plt.figure(figsize=(8,5))
sns.countplot(df['body-style'])
plt.show()

<h1 id="data-transformation">Transformação dos Dados</h1>

<h3>Avaliando os Dados faltantes</h3>

In [ ]:
df.isnull().sum().sum()

In [ ]:
missing_data = df.isnull()
missing_data.head()

<h3>Contando os valores faltantes</h3>

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

De acordo com o resultado acima, cada coluna tem 205 linhas de dados, 7 colunas tem dados faltantes:
<ol>
    <li>"normalized-losses": 41</li>
    <li>"num-of-doors": 2</li>
    <li>"bore": 4</li>
    <li>"stroke" : 4</li>
    <li>"horsepower": 2</li>
    <li>"peak-rpm": 2</li>
    <li>"price": 4</li>
</ol>

<h3>Tratando os dados faltantes</h3>
Existem diferentes maneiras de tratar os dados faltantes em um conjunto de dados. A opção por um determinado método é livre, porém, alguns métodos são mais indicados do que outros em diferentes situações. Aqui, vamos aplicar alguns métodos em diferentes colunas:

<b>Substituição pela média:</b>
<ul>
    <li>"normalized-losses": 41 missing data</li>
    <li>"stroke": 4 missing data</li>
    <li>"bore": 4 missing data</li>
    <li>"horsepower": 2 missing data</li>
    <li>"peak-rpm": 2 missing data</li>
</ul>

<b>Substituição pela maior frequência de ocorrência:</b>
<ul>
    <li>"num-of-doors": 2 missing data, substituir por "four".</li>
</ul>

<b>Exclusão da linha inteira:</b>
<ul>
    <li>"price": 4 missing data </li>
</ul>

<h4>Determinação da média das colunas</h4>

In [ ]:
avg_norm_loss = df["normalized-losses"].astype("float").mean(axis=0)
print("Average of normalized-losses:", avg_norm_loss)
avg_stroke = df['stroke'].astype('float').mean(axis=0)
print("Average of stroke:", avg_stroke)
avg_bore = df['bore'].astype('float').mean(axis=0)
print("Average of bore:", avg_bore)
avg_horsepower = df['horsepower'].astype('float').mean(axis=0)
print("Average horsepower:", avg_horsepower)
avg_peak_rpm = df['peak-rpm'].astype('float').mean(axis=0)
print("Average peak rpm:", avg_peak_rpm)

<h4>Substituição de "NaN" pela média</h4>

In [ ]:
df["normalized-losses"].replace(np.nan, avg_norm_loss, inplace=True)
df["stroke"].replace(np.nan, avg_stroke, inplace=True)
df["bore"].replace(np.nan, avg_bore, inplace=True)
df['horsepower'].replace(np.nan, avg_horsepower, inplace=True)
df['peak-rpm'].replace(np.nan, avg_peak_rpm, inplace=True)

<h4>Verificação da ocorrência de valores</h4>

In [ ]:
df['num-of-doors'].value_counts()

<h4>Substituição de "NaN" pela maior frequência</h4>

In [ ]:
df["num-of-doors"].replace(np.nan, "four", inplace=True)

<h4>Exclusão da linha inteira</h4>

In [ ]:
df.dropna(subset=["price"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isnull().sum().sum()

<h2 id="data_standardization">Padronização dos Dados</h2>
<p>
Os dados podem ser coletados por diferentes meios e em diferentes formatos. A padronização consiste em transformar os dados em um formato comum permitindo que os dados sejam comparados
</p>

<b>Exemplo</b>
<p>Transformar mpg para L/100km:</p>
<p>No nosso conjunto de dados, as colunas que informam o consumo de combustível "city-mpg" e "highway-mpg" são representados em unidade americanas (mpg - miles per gallon). Para melhor compreensão, vamos transformar para o padrão L/100km</p>

<p>A fórmula para conversão de unidades é<p>
L/100km = 235 / mpg


In [ ]:
# Converte mpg para L/100km
df['city-L/100km'] = 235/df["city-mpg"]
df['highway-L/100km'] = 235/df["highway-mpg"]

df.head()

<h2 id="data_normalization">Normalização dos Dados</h2>

<p>Alguns dados apresentam valores numéricos em escalas muito diferentes, o que dificulta a comparação. Para que estes dados possam ser comparados é realizada a normalização dos dados. Portanto, a normalização é o processo de transformar valores de variáveis em uma escala similar. Normalizações típicas incluem transformar os dados para uma escala de média 0, de variância 1, ou de intervalo entre 0 e 1.
</p>

<b>Exemplo</b>
<p>Neste exemplo, vamos transformar as colunas "length", "width" and "height" </p>
<p><b>Objetivo:</b> vamos normalizar estas variáveis para valores de 0 a 1.0</p>
<p><b>Abordagem:</b> substituir os valore originais por (valor original)/(valor máximo)</p>

In [ ]:
df['length'] = df['length']/df['length'].max()
df['width'] = df['width']/df['width'].max()
df['height'] = df['height']/df['height'].max()
df[['length','width', 'height']].head()